<a href="https://colab.research.google.com/github/rzqmhb/machine-learning-ti3f/blob/main/Minggu%2010%20-%20Recurrent%20Neural%20Network%20(RNN)/10_Roziq%20Mahbubi_RNN/RNN_praktikum2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Praktikum 2

###Setup

####Import TensorFlow

In [ ]:
import tensorflow as tf # import tensorflow
import numpy as np # import numpy
import os # import library untuk manipulasi tingkat OS
import time # import library waktu

####Download Dataset Shakespeare

In [ ]:
# Mengunduh file teks 'shakespeare.txt'
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


####Load Data

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Olah Teks

####Vectorize Teks

In [ ]:
# Membuat contoh teks dan memisahkan setiap karakter
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# Membuat layer StringLookup untuk mengubah karakter menjadi id unik
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [ ]:
# merubah id
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
# Membuat layer StringLookup lain untuk mengubah id kembali menjadi karakter
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
# merubah id
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# Menggabungkan karakter menjadi string
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
# Fungsi untuk mengubah ID menjadi teks
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

####Membuat Training Set dan Target

In [ ]:
# Mengubah semua teks menjadi ID
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
# Membuat dataset dari ID
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
# Mencetak beberapa ID pertama
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100 # mendefinisikan sequence

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # Membuat sequence dari ID

# Mencetak beberapa karakter pertama
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
# Mencetak beberapa text pertama
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
# Fungsi untuk membagi input dan target
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
# memecah teks menjadi karakter
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target) # Membuat dataset dari sequence

In [ ]:
# menampilkan sample data
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


####Membuat Batch Training

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Menyiapkan dataset untuk pelatihan
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

###Buat Model

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
# Membuat model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
# Membuat instance dari model
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

###Uji Model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([28, 38, 21,  0, 20, 56, 23, 36,  9, 16, 20, 17, 63, 24, 30, 61, 33,
       56, 58, 63, 61,  6, 57, 53, 23, 16, 50, 31,  0, 23, 24,  3, 46, 39,
       21, 25, 43, 65, 63, 35, 15, 62, 55, 37, 50, 61, 24, 28, 46, 40, 48,
       48, 30, 50, 32, 37, 54,  4,  1, 31, 34, 44, 23, 43, 57,  6, 23, 26,
       34, 32, 17, 38, 28, 17, 14, 37, 30, 32, 63, 20, 30, 38, 23, 47, 58,
       49, 32, 12, 47,  2, 63,  8, 16, 55, 48, 28, 33, 61, 44, 37])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"s death.\n\nQUEEN MARGARET:\nAnd here's to right our gentle-hearted king.\n\nYORK:\nOpen Thy gate of mercy"

Next Char Predictions:
 b"OYH[UNK]GqJW.CGDxKQvTqsxv'rnJCkR[UNK]JK!gZHLdzxVBwpXkvKOgaiiQkSXo$\nRUeJdr'JMUSDYODAXQSxGQYJhsjS;h x-CpiOTveX"


###Train Model

####Tambahan optimizer dan fungsi loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1866884, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

65.80451

In [ ]:
model.compile(optimizer='adam', loss=loss)

####Konfigurasi Checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

####Lakukan Proses Training

In [ ]:
EPOCHS = 20 # Menentukan jumlah epoch untuk pelatihan

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])  # Melakukan pelatihan model

Epoch 1/20
172/172 [==============================] - 15s 53ms/step - loss: 2.6961
Epoch 2/20
172/172 [==============================] - 12s 51ms/step - loss: 1.9703
Epoch 3/20
172/172 [==============================] - 10s 51ms/step - loss: 1.6924
Epoch 4/20
172/172 [==============================] - 11s 53ms/step - loss: 1.5347
Epoch 5/20
172/172 [==============================] - 12s 52ms/step - loss: 1.4382
Epoch 6/20
172/172 [==============================] - 11s 53ms/step - loss: 1.3714
Epoch 7/20
172/172 [==============================] - 11s 53ms/step - loss: 1.3201
Epoch 8/20
172/172 [==============================] - 11s 54ms/step - loss: 1.2755
Epoch 9/20
172/172 [==============================] - 11s 56ms/step - loss: 1.2352
Epoch 10/20
172/172 [==============================] - 11s 56ms/step - loss: 1.1947
Epoch 11/20
172/172 [==============================] - 11s 58ms/step - loss: 1.1552
Epoch 12/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1142
E

###Generate Teks

In [ ]:
# Membuat model untuk menghasilkan teks satu langkah pada satu waktu
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars) # Membuat instance dari model OneStep

In [ ]:
# Menghasilkan teks dengan model
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# Mencetak hasil dan waktu eksekusi
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:

Ghost of SABESEY:
You know the queen's pine; with robbers ready, impress'd
To have him more unclineded at many delicers
him.

NORTHUMBERLAND:
In his newge-brethern in the purts of France,
And from my business in the Capulets kind
Of every wreckned, with nothing fingers
To do thee speed to be a pover there.

Second Citizen:
Come on, sit by the sun,
Come on, with a bloody deed to my apparel;
But stay the hearts to me, and my life upon her;
Madam, your friends are brought my troth, I'll pay them.

KING LEWIS XI:
To wife, let thim be heard of that person.

GRUMIO:
Why for Claudio, sir?

ABRAHY:
No, my good lord Greecen:
I will to him in the common.

Second Servingman:
Here, sir.

First Senator:
Tell me, good naples, circling now!
And all the summers will entreat her good a
good favour.

COMINIUS:
Dost thou done?

ISABELLA:
It is a mind
of noble Geers! my boy shall serve a bawd:--for tears we still haded
withdrawing in the field?

WARWICK:
And I I have heard is gone; it is no blood,

In [ ]:
# Menghasilkan teks untuk beberapa instance sekaligus
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
# Mencetak hasil dan waktu eksekusi
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nSir, my reasons must not be confixed.\n\nKING RICHARD II:\nSo to offend him so dissembers, there a man\nFor making him with the racks. My gage\nNeckssities in him that's tooth'd; boy, in a fearful pow\ndeservent to beat my body's pabbia. What\nshould, no less, but there deny to decent\nOf our places of themselves: sir, were not before\nI call hom we will to do him right.\n\nROMEO:\nWhate'er it be told them most carelets that,\nGood patricy, and some of those loves.\nI, like a brother in the midule should\nBe subject: 'tis thursday neat, our kingdable dotary\nAs you have stop'd the suddent in the common bodies\nOf woltchmans have attendent you out:\nDo you cut with a man to men's place.\n\nGLOUCESTER:\nGood night to your recompense is ready; you are come\nTo hear it cannot hence to France to 'wis time\nwhat I can dream'd a word or thing\nBy heaven from too hit the heat of worthy:\nReshow me unthriver, and I knew not\nThat title can apporn the wreck, which I\nfear m

###Ekspor Model Generator

In [ ]:
# Menyimpan model
tf.saved_model.save(one_step_model, 'one_step')
# Memuat model yang telah disimpan
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
# Menghasilkan teks dengan model yang telah dimuat
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Who set his foungest toth mine eye seeming shuts
To bear the infinite doth character thieves.

Serv
